### SQLAlchemy

In [2]:
import pandas as pd
import requests 
import time
from sqlalchemy import *
# import MySQLdb

> 크롤링하는 py파일에 sqlalchemy의 engine연결 후 df.to_sql() 시도\
**-> sql 내 테이블은 만들어졌으나 아래와 같은 에러메세지, 데이터 들어가지지않음**

In [5]:
import configparser

config = configparser.ConfigParser()
config.read("../db.ini")
sql_datas = config["sql"]

In [7]:
%%writefile baemin_del_sqlalchemy.py

import pandas as pd
import requests 
import time
from sqlalchemy import *
import configparser

config = configparser.ConfigParser()
config.read("../db.ini")
sql_datas = config["sql"]

# 카테고리 수정 전(참고용)
# categories = {
#     100104 : "플라스틱용기",
#     100105 : "종이용기/박스",
#     100138 : "카페/음료용기",
#     100159 : "종이/비닐봉투",
#     100165 : "수저/위생용품",
#     100177 : "스티커/배너/포스터",
#     100184 : "포장재",
#     100188 : "기타가게용품",
# }

# 카테고리 수정 후 
categories = {
    100104 : "포장용품",
    100105 : "포장용품",
    100138 : "포장용품",
    100159 : "포장용품",
    100165 : "가게위생용품",
    100184 : "포장용품",
    100188 : "가게위생용품",
}


def baemin_items(category, page):
    url = "https://mart.baemin.com/api/v2/front/categories/goods/{}/paging?goodsSortType=BASIC&page={}".format(category, page)
    response = requests.get(url)
    elements = response.json()['data']['simpleGoodsDtoPage']['content']
    datas = []
    for element in elements:
        title = element['name']
        price = element['goodsPrice']
#         o_price = element['customerPrice']
        # 원인은 모르겠지만.. 100050/밀가루 쪽에서 에러남
        try :
            min_gram = element['sizeDesc']
        except:
            min_gram = "-"
        try:
            price_per_gram = element['priceDesc']
        except:
            price_per_gram = "-"
        link = "https://mart.baemin.com/goods/detail/" + str(element['id'])

        datas.append({
            "category": categories[category],
            "title" : title,
            "price" : price,
#             "o_price" : o_price,
            "min_gram" : min_gram,
            "price_per_gram" : price_per_gram,
            "link" : link,
        })
        
    return pd.DataFrame(datas)

dfs = []
for category in categories:
    print(categories[category])
    for page in range(12): # url에서 page가 0부터 시작
        print(page, end=" ")
        df = baemin_items(category, page)
        dfs.append(df)
    print()
    time.sleep(2)

baemin_mart_delivery_goods_df = pd.concat(dfs, ignore_index=True)
baemin_mart_delivery_goods_df

engine = create_engine("mysql://{}:{}@{}/test?charset=utf8"
                       .format(sql_datas["user"], sql_datas["pw"], sql_datas["public_ip"]), encoding='utf8')
baemin_mart_delivery_goods_df.to_sql(name="baemin_del_goods", con=engine, if_exists="replace")

# baemin_mart_delivery_goods_df.to_csv("baemin_mart_delivery_goods.csv", index=False, encoding="utf-8-sig")

# print("="*100)
# print("It's saved as baemin_mart_delivery_goods.csv file.")
# print("="*100)


Writing baemin_del_sqlalchemy.py


> 초반 데이터베이스 생성 시 latin으로 되어있어서 utf8로 변경해줬어야 되는 오류였다.

In [10]:
# %load baemin_del_sqlalchemy.py

import pandas as pd
import requests 
import time
from sqlalchemy import *
import configparser

config = configparser.ConfigParser()
config.read("../db.ini")
sql_datas = config["sql"]

# 카테고리 수정 전(참고용)
# categories = {
#     100104 : "플라스틱용기",
#     100105 : "종이용기/박스",
#     100138 : "카페/음료용기",
#     100159 : "종이/비닐봉투",
#     100165 : "수저/위생용품",
#     100177 : "스티커/배너/포스터",
#     100184 : "포장재",
#     100188 : "기타가게용품",
# }

# 카테고리 수정 후 
categories = {
    100104 : "포장용품",
    100105 : "포장용품",
    100138 : "포장용품",
    100159 : "포장용품",
    100165 : "가게위생용품",
    100184 : "포장용품",
    100188 : "가게위생용품",
}


def baemin_items(category, page):
    url = "https://mart.baemin.com/api/v2/front/categories/goods/{}/paging?goodsSortType=BASIC&page={}".format(category, page)
    response = requests.get(url)
    elements = response.json()['data']['simpleGoodsDtoPage']['content']
    datas = []
    for element in elements:
        title = element['name']
        price = element['goodsPrice']
#         o_price = element['customerPrice']
        # 원인은 모르겠지만.. 100050/밀가루 쪽에서 에러남
        try :
            min_gram = element['sizeDesc']
        except:
            min_gram = "-"
        try:
            price_per_gram = element['priceDesc']
        except:
            price_per_gram = "-"
        link = "https://mart.baemin.com/goods/detail/" + str(element['id'])

        datas.append({
            "category": categories[category],
            "title" : title,
            "price" : price,
#             "o_price" : o_price,
            "min_gram" : min_gram,
            "price_per_gram" : price_per_gram,
            "link" : link,
        })
        
    return pd.DataFrame(datas)

dfs = []
for category in categories:
    print(categories[category])
    for page in range(12): # url에서 page가 0부터 시작
        print(page, end=" ")
        df = baemin_items(category, page)
        dfs.append(df)
    print()
    time.sleep(2)

baemin_mart_delivery_goods_df = pd.concat(dfs, ignore_index=True)
baemin_mart_delivery_goods_df

engine = create_engine("mysql://{}:{}@{}/test?charset=utf8"
                       .format(sql_datas["user"], sql_datas["pw"], sql_datas["public_ip"]), encoding='utf8')
baemin_mart_delivery_goods_df.to_sql(name="baemin_del_goods", con=engine, if_exists="replace")

# baemin_mart_delivery_goods_df.to_csv("baemin_mart_delivery_goods.csv", index=False, encoding="utf-8-sig")

# print("="*100)
# print("It's saved as baemin_mart_delivery_goods.csv file.")
# print("="*100)


포장용품
0 1 2 3 4 5 6 7 8 9 10 11 
포장용품
0 1 2 3 4 5 6 7 8 9 10 11 
포장용품
0 1 2 3 4 5 6 7 8 9 10 11 
포장용품
0 1 2 3 4 5 6 7 8 9 10 11 
가게위생용품
0 1 2 3 4 5 6 7 8 9 10 11 
포장용품
0 1 2 3 4 5 6 7 8 9 10 11 
가게위생용품
0 1 2 3 4 5 6 7 8 9 10 11 


In [11]:
baemin_mart_delivery_goods_df

,category,title,price,min_gram,price_per_gram,link
0,포장용품,탕용기(소),57000,225Øx85(mm),개당 285원 / 200ea,https://mart.baemin.com/goods/detail/585
1,포장용품,[소량] 탕용기(소),29700,225Øx85(mm),개당 297원 / 100ea,https://mart.baemin.com/goods/detail/1778
2,포장용품,친환경 탕용기(소),30300,225Øx80(mm),개당 303원 / 100ea,https://mart.baemin.com/goods/detail/7446
3,포장용품,친환경 탕용기(소/JH),53500,220Øx82(mm),개당 268원 / 200ea,https://mart.baemin.com/goods/detail/9285
4,포장용품,친환경 탕용기(중/JH),57000,220Øx103(mm),개당 285원 / 200ea,https://mart.baemin.com/goods/detail/9286
...,...,...,...,...,...,...
2390,가게위생용품,[든든]리오 락스 2L,1050,2L,"개당 1,050원 / 1ea",https://mart.baemin.com/goods/detail/8833
2391,가게위생용품,[든든]스카치브라이트 수세미(다목적용_청색) 2 개,1980,2ea,"개당 1,980원 / 1ea",https://mart.baemin.com/goods/detail/8828
2392,가게위생용품,[든든]크린업 부직포행주(삼색_38*38cm) 3 개,770,3ea,개당 770원 / 1ea,https://mart.baemin.com/goods/detail/8826
2393,가게위생용품,[든든]크린업 스폰지수세미(15*9*3cm) 2 개,1170,2ea,"개당 1,170원 / 1ea",https://mart.baemin.com/goods/detail/8823


### MongoDB

In [34]:
%%writefile baemin_del_mongodb.py

import pandas as pd
import requests 
import time
import pymongo
import configparser

config = configparser.ConfigParser()
config.read("../db.ini")
mongodb = config["mongodb"]

categories = {
    100104 : "포장용품",
    100105 : "포장용품",
    100138 : "포장용품",
    100159 : "포장용품",
    100165 : "가게위생용품",
    100184 : "포장용품",
    100188 : "가게위생용품",
}


def baemin_items(category, page):
    url = "https://mart.baemin.com/api/v2/front/categories/goods/{}/paging?goodsSortType=BASIC&page={}".format(category, page)
    response = requests.get(url)
    elements = response.json()['data']['simpleGoodsDtoPage']['content']
    datas = []
    for element in elements:
        title = element['name']
        price = element['goodsPrice']
#         o_price = element['customerPrice']
        # 원인은 모르겠지만.. 100050/밀가루 쪽에서 에러남
        try :
            min_gram = element['sizeDesc']
        except:
            min_gram = "-"
        try:
            price_per_gram = element['priceDesc']
        except:
            price_per_gram = "-"
        link = "https://mart.baemin.com/goods/detail/" + str(element['id'])

        datas.append({
            "category": categories[category],
            "title" : title,
            "price" : price,
#             "o_price" : o_price,
            "min_gram" : min_gram,
            "price_per_gram" : price_per_gram,
            "link" : link,
        })
        
    return pd.DataFrame(datas)

dfs = []
for category in categories:
    print(categories[category])
    for page in range(12): # url에서 page가 0부터 시작
        print(page, end=" ")
        df = baemin_items(category, page)
        dfs.append(df)
    print()
    time.sleep(2)

baemin_mart_delivery_goods_df = pd.concat(dfs, ignore_index=True)
baemin_mart_delivery_goods_df

client = pymongo.MongoClient("mongodb://{}:{}@{}:27017/".format(mongodb["user"], mongodb["pw"], mongodb["public_ip"]))
baemin_del = client.baemin_db.baemin_del
items = baemin_mart_delivery_goods_df.to_dict('records')
baemin_del.insert(items)

# baemin_mart_delivery_goods_df.to_csv("baemin_mart_delivery_goods.csv", index=False, encoding="utf-8-sig")

print("="*60)
print("baemin data is in MongoDB..")
print("="*60)


Overwriting baemin_del_mongodb.py


In [32]:
client = pymongo.MongoClient("mongodb://{}:{}@{}:27017/".format(mongodb["user"], mongodb["pw"], mongodb["public_ip"]))
baemin_del = client.baemin_db.baemin_del
items = baemin_mart_delivery_goods_df.to_dict('records')
log_data = baemin_del.insert(items)
len(log_data)

2395

In [ ]:
# %load baemin_del_mongodb.py

import pandas as pd
import requests 
import time
import pymongo
import configparser

config = configparser.ConfigParser()
config.read("../db.ini")
mongodb = config["mongodb"]

categories = {
    100104 : "포장용품",
    100105 : "포장용품",
    100138 : "포장용품",
    100159 : "포장용품",
    100165 : "가게위생용품",
    100184 : "포장용품",
    100188 : "가게위생용품",
}


def baemin_items(category, page):
    url = "https://mart.baemin.com/api/v2/front/categories/goods/{}/paging?goodsSortType=BASIC&page={}".format(category, page)
    response = requests.get(url)
    elements = response.json()['data']['simpleGoodsDtoPage']['content']
    datas = []
    for element in elements:
        title = element['name']
        price = element['goodsPrice']
#         o_price = element['customerPrice']
        # 원인은 모르겠지만.. 100050/밀가루 쪽에서 에러남
        try :
            min_gram = element['sizeDesc']
        except:
            min_gram = "-"
        try:
            price_per_gram = element['priceDesc']
        except:
            price_per_gram = "-"
        link = "https://mart.baemin.com/goods/detail/" + str(element['id'])

        datas.append({
            "category": categories[category],
            "title" : title,
            "price" : price,
#             "o_price" : o_price,
            "min_gram" : min_gram,
            "price_per_gram" : price_per_gram,
            "link" : link,
        })
        
    return pd.DataFrame(datas)

dfs = []
for category in categories:
    print(categories[category])
    for page in range(12): # url에서 page가 0부터 시작
        print(page, end=" ")
        df = baemin_items(category, page)
        dfs.append(df)
    print()
    time.sleep(2)

baemin_mart_delivery_goods_df = pd.concat(dfs, ignore_index=True)
baemin_mart_delivery_goods_df

client = pymongo.MongoClient("mongodb://{}:{}@{}:27017/".format(mongodb["user"], mongodb["pw"], mongodb["public_ip"]))
baemin_del = client.baemin_db.baemin_del
items = baemin_mart_delivery_goods_df.to_dict('records')
baemin_del.insert(items)

# baemin_mart_delivery_goods_df.to_csv("baemin_mart_delivery_goods.csv", index=False, encoding="utf-8-sig")

print("="*60)
print("baemin data is in MongoDB..")
print("="*60)


> 크롤링, scv저장, DB 저장(sql, mongodb) 합치기

## baemin_class

In [5]:
%%writefile baemin_class.py

import pandas as pd
import requests
import time
import configparser
import pymongo
from sqlalchemy import create_engine


config = configparser.ConfigParser()
config.read("../db.ini")
sql_datas = config["sql"]
mongodb = config["mongodb"]

categories = {
    100015: "냉동상품",
    100024: "냉장식품",
    100028: "유제품",
    100032: "식용유/설탕/소금",
    100038: "장류/조미료",
    100043: "소스/양념/육수",
    100050: "밀가루/라면/믹스류",
    100054: "편의식/가공기타",
    100061: "반찬류",
    100067: "야채/채소",
    100077: "과일",
    100081: "쌀/잡곡/견과",
    100085: "수산/건어물",
    100096: "축산/계란",
    100104: "포장용품",
    100105: "포장용품",
    100138: "포장용품",
    100159: "포장용품",
    100165: "가게위생용품",
    100184: "포장용품",
    100188: "가게위생용품",
}

class Baemin:
        
#     def __init__(self):
        
               
    def __baemin_items(self, category, page):
        url = "https://mart.baemin.com/api/v2/front/categories/goods/{}/paging?goodsSortType=BASIC&page={}".format(
            self.category, self.page)
        response = requests.get(url)
        elements = response.json()['data']['simpleGoodsDtoPage']['content']
        datas = []
        for element in elements:
            title = element['name']
            price = element['goodsPrice']
            # 원인은 모르겠지만.. 100050/밀가루 쪽에서 에러남
            try:
                min_gram = element['sizeDesc']
            except:
                min_gram = "-"
            try:
                price_per_gram = element['priceDesc']
            except:
                price_per_gram = "-"
            link = "https://mart.baemin.com/goods/detail/" + str(element['id'])

            datas.append({
                "category": categories[category],
                "title": title,
                "price": price,
                "min_gram": min_gram,
                "price_per_gram": price_per_gram,
                "link": link,
            })

        return pd.DataFrame(datas)
    
    
    def __last_page(self, category):
        url = "https://mart.baemin.com/api/v2/front/categories/goods/{}/paging?goodsSortType=BASIC&page=0".format(
        self.category)
        response = requests.get(url)
        max_pages = response.json()['data']['simpleGoodsDtoPage']['totalPages']
        return max_pages
         

    def crawling_baemin(self, categories, page=0, csv=False, sql=False, mongodb=False):
        dfs = []
        for self.category in categories:
            max_pages = self.__last_page(self.category)
            print("품목 : {}, 총페이지 : {} ".format(categories[self.category], max_pages))
            for self.page in range(max_pages): # url에서 page가 0부터 시작
                print(self.page + 1, end=" ")
                df = self.__baemin_items(self.category, self.page)
                dfs.append(df)
            print()
            time.sleep(2)
        global baemin_df
        baemin_df = pd.concat(dfs, ignore_index=True)
        print("finish crwaling_baemin")
        
        if csv:
            self.__save_csv()
        elif sql:
            self.__db_sql()
        elif mongodb:
            self.__db_mongodb()
        else:
            return baemin_df


    def __save_csv(self):
        baemin_df.to_csv("baemin_df.csv", index=False, encoding="utf-8-sig")
        print("="*60)
        print("baemin data save baemin_df.csv..")
        print("="*60)


    def __db_sql(self):
        engine = create_engine("mysql://{}:{}@{}/test?charset=utf8"
                               .format(sql_datas["user"], sql_datas["pw"], sql_datas["public_ip"]), encoding='utf8')
        baemin_df.to_sql(name="baemin_items", con=engine, if_exists="replace")
        print("="*60)
        print("baemin data is in sql..")
        print("="*60)


    def __db_mongodb(self):
        client = pymongo.MongoClient(
            "mongodb://{}:{}@{}:27017/".format(mongodb["user"], mongodb["pw"], mongodb["public_ip"]))
        baemin_items = client.baemin_db.baemin_items
        items = baemin_df.to_dict('records')
        baemin_items.insert(items)
        print("="*60)
        print("baemin data is in MongoDB..")
        print("="*60)


Overwriting baemin_class.py


In [6]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [7]:
%whos

Interactive namespace is empty.


In [8]:
import baemin_class as bm

In [9]:
baemin = bm.Baemin()

In [10]:
baemin.crawling_baemin(bm.categories)

품목 : 냉동상품, 총페이지 : 8 
1 2 3 4 5 6 7 8 
품목 : 냉장식품, 총페이지 : 1 
1 
품목 : 유제품, 총페이지 : 1 
1 
품목 : 식용유/설탕/소금, 총페이지 : 1 
1 
finish crwaling_baemin


,category,title,price,min_gram,price_per_gram,link
0,냉동상품,[유통기한임박] [든든] 주원산오리 매운오리불고기 800g 5개,34500,5개,"개당 6,900원 / 5ea",https://mart.baemin.com/goods/detail/15168
1,냉동상품,[든든] 돼지양념후라이드 1kg 6개,44200,1kg,"1kg 당 7,367원 / 6ea",https://mart.baemin.com/goods/detail/15135
2,냉동상품,[든든] 엄지척 김치 왕교자 1.05kg,5740,1.05kg,"개당 5,740원 / 1ea",https://mart.baemin.com/goods/detail/15024
3,냉동상품,[든든] 엄지척 왕교자 1.05kg,5610,1.05kg,"개당 5,610원 / 1ea",https://mart.baemin.com/goods/detail/15023
4,냉동상품,[든든] 살사 세모 군만두 1kg,5840,1kg,"개당 5,840원 / 1ea",https://mart.baemin.com/goods/detail/15022
...,...,...,...,...,...,...
995,식용유/설탕/소금,[든든] 식자재왕 참깨향기름_골드 1.8L,10500,1.8L,"개당 10,500원 / 1ea",https://mart.baemin.com/goods/detail/9621
996,식용유/설탕/소금,[든든]이츠웰 NEW 맥아물엿 15kg,25590,15kg,"개당 25,590원 / 1ea",https://mart.baemin.com/goods/detail/8346
997,식용유/설탕/소금,[든든] 식자재왕 이온물엿 8kg,8610,8kg,"개당 8,610원 / 1ea",https://mart.baemin.com/goods/detail/9609
998,식용유/설탕/소금,[든든]이츠웰 들기름 1.65kg,20910,1.65kg,"개당 20,910원 / 1ea",https://mart.baemin.com/goods/detail/8330


In [11]:
baemin.crawling_baemin(bm.categories, csv=True)

품목 : 냉동상품, 총페이지 : 8 
1 2 3 4 5 6 7 8 
품목 : 냉장식품, 총페이지 : 1 
1 
품목 : 유제품, 총페이지 : 1 
1 
품목 : 식용유/설탕/소금, 총페이지 : 1 
1 
finish crwaling_baemin
baemin data save baemin_df.csv..


In [12]:
baemin.crawling_baemin(bm.categories, sql=True)

품목 : 냉동상품, 총페이지 : 8 
1 2 3 4 5 6 7 8 
품목 : 냉장식품, 총페이지 : 1 
1 
품목 : 유제품, 총페이지 : 1 
1 
품목 : 식용유/설탕/소금, 총페이지 : 1 
1 
finish crwaling_baemin
baemin data is in sql..


In [13]:
baemin.crawling_baemin(bm.categories, mongodb=True)

품목 : 냉동상품, 총페이지 : 8 
1 2 3 4 5 6 7 8 
품목 : 냉장식품, 총페이지 : 1 
1 
품목 : 유제품, 총페이지 : 1 
1 
품목 : 식용유/설탕/소금, 총페이지 : 1 
1 
finish crwaling_baemin
baemin data is in MongoDB..
